In [ ]:
# default_exp core

# Node_cluster

> Basic classes to generate the cluster_node clas 


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
import sklearn
from sklearn import cluster, datasets
import anytree
from shapely.geometry import LineString
from shapely.ops import polygonize, cascaded_union
from shapely.geometry import box
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import polygonize_full, linemerge, unary_union
from matplotlib import cm
from anytree import NodeMixin, RenderTree

## cluster
A basic cluster definition 

In [ ]:
#export 
class cluster(object):
    polygon_cluster = None
    point_cluster_noise = None

    def __init__(self):
        self.polygon_cluster = None
        self.density = None
        self.point_cluster_noise = None
    